In [1]:
## Installing the dependencies (might take some time)

!pip install -q pytesseract
!sudo apt install  -q tesseract-ocr
!pip install  -q transformers
!pip install  -q pytorch-lightning
!pip install  -q einops
!pip install  -q tqdm
!pip install  -q 'Pillow==7.1.2'
!pip install -q wandb
!pip install -q torchmetrics

Reading package lists...
Building dependency tree...
Reading state information...
tesseract-ocr is already the newest version (4.1.1-2build2).
0 upgraded, 0 newly installed, 0 to remove and 93 not upgraded.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
beatrix-jupyterlab 3.1.7 requires google-cloud-bigquery-storage, which is not installed.
pytesseract 0.3.10 requires Pillow>=8.0.0, but you have pillow 7.1.2 which is incompatible.
imageio 2.19.3 requires pillow>=8.3.2, but you have pillow 7.1.2 which is incompatible.


In [2]:
## Cloning the repository
!git clone https://github.com/uakarsh/LiLT.git

Cloning into 'LiLT'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 34 (delta 10), reused 25 (delta 4), pack-reused 0
Unpacking objects: 100% (34/34), 277.70 KiB | 1.15 MiB/s, done.


In [3]:
## Logging into wandb

import wandb
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb_api")
wandb.login(key=secret_value_0)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
import pandas as pd
import os
from PIL import Image, ImageDraw
from transformers import RobertaTokenizer
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import pytorch_lightning as pl

## Adding the path of lilt to system path
import sys
sys.path.append('./LiLT/src/lilt/')

from dataset import create_features
from modeling import LiLT

In [5]:
## Hyperparameters
seed = 42

## One can change this configuration and try out new combination
config = {
  "hidden_dropout_prob": 0.1,
  "hidden_size_t": 768,
  "hidden_size" : 768,
  "hidden_size_l": 768 // 6,
  "intermediate_ff_size_factor": 4,
  "max_2d_position_embeddings": 1001,
  "max_seq_len_l": 512,
  "max_seq_len_t" : 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  'dim_head' : 64,
  "shape_size": 96,
  "vocab_size": 50265,
  "eps": 1e-12,
  "fine_tune" : True
}

## A Small set of data for training and testing LiLT

In [6]:
from tqdm.auto import tqdm

## For the purpose of prediction
id2label = []
label2id = {}

curr_class = 0
## Preparing the Dataset
base_directory = '../input/the-rvlcdip-dataset-test/test'
dict_of_img_labels = {'img':[], 'label':[]}

max_sample_per_class = 250

for label in tqdm(os.listdir(base_directory)):
    img_path = os.path.join(base_directory, label)
    
    count = 0
    if label not in label2id:
        label2id[label] = curr_class
        curr_class+=1
        id2label.append(label)
        
    for img in os.listdir(img_path):
        if count>max_sample_per_class:
            break
            
        curr_img_path = os.path.join(img_path, img)
        dict_of_img_labels['img'].append(curr_img_path)
        dict_of_img_labels['label'].append(label2id[label])
        count+=1

  0%|          | 0/16 [00:00<?, ?it/s]

In [7]:
import pandas as pd
df = pd.DataFrame(dict_of_img_labels)

In [8]:
from sklearn.model_selection import train_test_split as tts
train_df, valid_df = tts(df, random_state = seed, stratify = df['label'], shuffle = True)
train_df = train_df.reset_index().drop(columns = ['index'], axis = 1)
valid_df = valid_df.reset_index().drop(columns = ['index'], axis = 1)

In [9]:
## Creating the dataset

class RVLCDIPData(Dataset):
    
    def __init__(self, image_list, label_list, tokenizer, max_len = 512, size = 1000):
        
        self.image_list = image_list
        self.label_list = label_list
        self.tokenizer = tokenizer
        self.max_seq_length = max_len
        self.size = size
        
    def __len__(self):
        return len(self.image_list)
    
    def __getitem__(self, idx):
        img_path = self.image_list[idx]
        label = self.label_list[idx]
        
        boxes, words, normal_box = create_features(
                                img_path = img_path,
                                tokenizer = self.tokenizer,
                                max_seq_length = self.max_seq_length,
                                size = self.size,
                                use_ocr  = True,
                               )
        
        final_encoding = {'input_boxes': boxes, 'input_words': words}
        final_encoding['label'] = torch.as_tensor(label).long()
        
        return final_encoding

In [10]:
## Defining tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

## Defining the dataset
train_ds = RVLCDIPData(train_df['img'].tolist(), train_df['label'].tolist(),
                      tokenizer, config['max_seq_len_l'])
val_ds = RVLCDIPData(valid_df['img'].tolist(), valid_df['label'].tolist(),
                      tokenizer,config['max_seq_len_l'])

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [11]:
def collate_fn(list_of_ds):
  encoding = {'input_boxes' : [], 'input_words' : [],'label' : []}

  for i in list_of_ds:
    for key in encoding.keys():
      encoding[key].append(i[key])
    
  for key in encoding.keys():
    encoding[key] = torch.stack(encoding[key], axis = 0)
    
  return encoding

## Defining the data module

In [12]:
class DataModule(pl.LightningDataModule):

  def __init__(self, train_dataset, val_dataset,  batch_size = 4):

    super(DataModule, self).__init__()
    self.save_hyperparameters()
    self.train_dataset = train_dataset
    self.val_dataset = val_dataset
    self.batch_size = batch_size

  def train_dataloader(self):
    return DataLoader(self.train_dataset, batch_size = self.batch_size | self.hparams.batch_size, 
                      collate_fn = collate_fn, shuffle = True, num_workers = 2)
  
  def val_dataloader(self):
    return DataLoader(self.val_dataset, batch_size = self.batch_size | self.hparams.batch_size,
                                  collate_fn = collate_fn, shuffle = False, num_workers = 2)

In [13]:
class LiLTForClassification(nn.Module):
  
    def __init__(self, config):
      super(LiLTForClassification, self).__init__()

      self.lilt = LiLT(config)
      self.config = config
      self.dropout = nn.Dropout(config['hidden_dropout_prob'])
      self.linear_layer = nn.Linear(in_features = config['hidden_size'] * 2, out_features = len(id2label))  ## Number of Classes

    def forward(self, batch_dict):
        encodings = self.lilt(batch_dict['input_words'], batch_dict['input_boxes'])
        final_out = torch.cat([encodings['layout_hidden_states'][-1], 
                              encodings['text_hidden_states'][-1]
                              ],
                             axis = -1)[:, 0, :]
        final_out = self.linear_layer(final_out)
        return final_out

In [14]:
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# lilt = LiLTForClassification(config).to(device)
# datamodule = DataModule(train_ds, val_ds)

# sample = next(iter(datamodule.train_dataloader()))
# encoding = lilt(sample)

In [15]:
## Defining pytorch lightning model
from sklearn.metrics import accuracy_score, confusion_matrix
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import torchmetrics

class LiLTPL(pl.LightningModule):

  def __init__(self, config , lr = 5e-5):
    super(LiLTPL, self).__init__()
    
    self.save_hyperparameters()
    self.config = config
    self.lilt = LiLTForClassification(config)
    
    self.num_classes = len(id2label)
    self.train_accuracy_metric = torchmetrics.Accuracy()
    self.val_accuracy_metric = torchmetrics.Accuracy()
    self.f1_metric = torchmetrics.F1Score(num_classes=self.num_classes)
    self.precision_macro_metric = torchmetrics.Precision(
            average="macro", num_classes=self.num_classes
        )
    self.recall_macro_metric = torchmetrics.Recall(
            average="macro", num_classes=self.num_classes
        )
    self.precision_micro_metric = torchmetrics.Precision(average="micro")
    self.recall_micro_metric = torchmetrics.Recall(average="micro")

  def forward(self, batch_dict):
    logits = self.lilt(batch_dict)
    return logits

  def training_step(self, batch, batch_idx):
    logits = self.forward(batch)

    loss = nn.CrossEntropyLoss()(logits, batch['label'])
    preds = torch.argmax(logits, 1)

    ## Calculating the accuracy score
    train_acc = self.train_accuracy_metric(preds, batch["label"])

    ## Logging
    self.log('train/loss', loss,prog_bar = True, on_epoch=True, logger=True, on_step=True)
    self.log('train/acc', train_acc, prog_bar = True, on_epoch=True, logger=True, on_step=True)

    return loss
  
  def validation_step(self, batch, batch_idx):
    logits = self.forward(batch)
    loss = nn.CrossEntropyLoss()(logits, batch['label'])
    preds = torch.argmax(logits, 1)
    
    labels = batch['label']
    # Metrics
    valid_acc = self.val_accuracy_metric(preds, labels)
    precision_macro = self.precision_macro_metric(preds, labels)
    recall_macro = self.recall_macro_metric(preds, labels)
    precision_micro = self.precision_micro_metric(preds, labels)
    recall_micro = self.recall_micro_metric(preds, labels)
    f1 = self.f1_metric(preds, labels)

    # Logging metrics
    self.log("valid/loss", loss, prog_bar=True, on_step=True, logger=True)
    self.log("valid/acc", valid_acc, prog_bar=True, on_epoch=True, logger=True, on_step=True)
    self.log("valid/precision_macro", precision_macro, prog_bar=True, on_epoch=True, logger=True, on_step=True)
    self.log("valid/recall_macro", recall_macro, prog_bar=True, on_epoch=True, logger=True, on_step=True)
    self.log("valid/precision_micro", precision_micro, prog_bar=True, on_epoch=True, logger=True, on_step=True)
    self.log("valid/recall_micro", recall_micro, prog_bar=True, on_epoch=True, logger=True, on_step=True)
    self.log("valid/f1", f1, prog_bar=True, on_epoch=True)
    
    return {"label": batch['label'], "logits": logits}

  def validation_epoch_end(self, outputs):
        labels = torch.cat([x["label"] for x in outputs])
        logits = torch.cat([x["logits"] for x in outputs])
        preds = torch.argmax(logits, 1)

        wandb.log({"cm": wandb.sklearn.plot_confusion_matrix(labels.cpu().numpy(), preds.cpu().numpy())})
        self.logger.experiment.log(
            {"roc": wandb.plot.roc_curve(labels.cpu().numpy(), logits.cpu().numpy())}
        )
        
  def configure_optimizers(self):
    return torch.optim.AdamW(self.parameters(), lr = self.hparams['lr'])

In [16]:
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger

def main():
    datamodule = DataModule(train_ds, val_ds)
    lilt_pl = LiLTPL(config)

    checkpoint_callback = ModelCheckpoint(
        dirpath="./models", monitor="valid/loss", mode="min"
    )
    
    wandb.init(config=config, project="RVL CDIP with LiLT")
    wandb_logger = WandbLogger(project="RVL CDIP with LiLT", entity="iakarshu")
    
    pl.seed_everything(seed, workers=True)
    trainer = pl.Trainer(
        default_root_dir="logs",
        accelerator = 'gpu',
        devices = 'auto',
        max_epochs=2,
#         fast_dev_run=False,
        logger=wandb_logger,
        callbacks=[checkpoint_callback],
#         deterministic=True,
#         auto_scale_batch_size = 'power'
    )
#     print("Tuning the batch size...")
    # Find the batch size
#     trainer.tune(lilt_pl, datamodule)
    print("The optimal batch size is:", datamodule.hparams.batch_size)
    trainer.fit(lilt_pl, datamodule)

In [17]:
if __name__ == "__main__":
    main()

wandb: Currently logged in as: iakarshu. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.7/site-packages/pytorch_lightning/loggers/wandb.py:353: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  "There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse"


The optimal batch size is: 4


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]